In [1]:
import sys
import pyccl as ccl
import pickle
import camb
sys.path.insert(0,'./')
from importlib import reload
from astropy.cosmology import Planck15 as cosmo
from astropy.cosmology import default_cosmology
from astropy import units
import astropy

In [2]:
from cov_3X2 import *
from fisher import *

In [3]:
import lsst_utils
reload(lsst_utils)
from lsst_utils import *

In [ ]:
fig_home='./figures/'
fig_format='pdf'

In [4]:
def cosmo_w0_wa(cosmo=None,w0=-1,wa=0):
    attrs=['H0','Om0', 'Ode0','Tcmb0', 'Neff', 'm_nu', 'Ob0']
    args={}
    args['w0']=w0
    args['wa']=wa
    for a in attrs:
        args[a]=getattr(cosmo,a)
    cosmo_w=astropy.cosmology.w0waCDM(**args)
    return cosmo_w
cosmo_w0_wa(cosmo=cosmo)

w0waCDM(H0=67.7 km / (Mpc s), Om0=0.307, Ode0=0.691, w0=-1, wa=0, Tcmb0=2.725 K, Neff=3.05, m_nu=[0.   0.   0.06] eV, Ob0=0.0486)

In [5]:
def cosmo_h_set(cosmo=None,cosmo_params={}):
    if cosmo_params['wa']!=0:
        cosmo=cosmo_w0_wa(cosmo=cosmo,w0=cosmo_params['w'],wa=cosmo_params['wa'])
    cosmo2=cosmo.clone(H0=cosmo_params['h']*100,
                        Om0=cosmo_params['Om'],
                       Ob0=cosmo_params['Omb'],
#                        Odm0=cosmo_params['Omd'],
#                        Ok0=cosmo_params['Omk'],
#                        w=cosmo_params['w'],
                       m_nu=[0,0,cosmo_params['mnu']]*units.eV
                    )

In [6]:
def fisher_calc(params=['As'],Nx=3,dx_max=0.01,do_log=False,kappa_class=None,do_cov=False,
               cov_3X2_kwargs={},zs_bins_kwargs={}):
    zs_bins=lsst_source_tomo_bins(**zs_bins_kwargs)
    kappa_class=cov_3X2(zs_bins=zs_bins,**cov_3X2_kwargs)
    
    cosmo_fid=kappa_class.Ang_PS.PS.cosmo_params.copy()
    cosmo_h=kappa_class.Ang_PS.PS.cosmo_h.clone()
    
    cl0G=kappa_class.cl_tomo()
    cl_t=cl0G['stack'].compute()
    cov=cl_t['cov']
    kappa_class.Ang_PS.reset()
    do_cov0=np.copy(kappa_class.do_cov)
    kappa_class.do_cov=do_cov

    Dx=np.linspace((1-dx_max),(1+dx_max),Nx)
    ndim=len(params)
    
    x_vars={}
    models={}
    model_derivs={}
    covs={}
    cov_derivs={}
    for p in params:
        x0=cosmo_fid[p]
        if do_log:
            x0=np.absolute(x0)
            x_vars[p]=x0**Dx
            if x0==1:
                x_vars[p]=(2.**Dx)/2. # 1**x=1
            if x0==0:
                x_vars[p]=np.logspace(-3,-2,Nx)
            x_vars[p]*=np.sign(cosmo_fid[p])
            x_grad=np.gradient(np.log(x_vars[p]))
        else:
            x_vars[p]=x0*Dx #np.linspace(x0*(1-dx_max),x0*(1+dx_max),Nx)
            if x0==0:
                x_vars[p]=Dx-1
            x_grad=np.gradient(x_vars[p])
        
        models[p]={}
        covs[p]={}
#         print(p,x_vars[p])
        for i in np.arange(Nx):
            cosmo_t=cosmo_fid.copy()
         
            cosmo_t[p]=x_vars[p][i]
            
            if p=='s8':
                s80=cosmo_fid['s8']
                cosmo_t['As']*=(cosmo_t['s8']/s80)**2
                kappa_class.Ang_PS.PS.get_pk(z=[0],cosmo_params=cosmo_t,return_s8=True)
                kappa_class.Ang_PS.PS.pk=None
                x_vars[p][i]=kappa_class.Ang_PS.PS.s8
#                 print(x_vars[p][i],s80,cosmo_t['s8'])
            cosmo_h2=cosmo_h_set(cosmo=cosmo,cosmo_params=cosmo_t)
            cl0G=kappa_class.cl_tomo(cosmo_params=cosmo_t,cosmo_h=cosmo_h2)
            cl_t=cl0G['stack'].compute()
            models[p][i]=cl_t['cl']
            covs[p][i]=cl_t['cov']
            kappa_class.Ang_PS.reset()
            
        model_derivs[p]=np.gradient(np.array([models[p][i] for i in np.arange(Nx)]),axis=0).T
        model_derivs[p]/=x_grad
        model_derivs[p]=model_derivs[p][:,np.int(Nx/2)]
        
        if do_cov:
            cov_derivs[p]=np.gradient(np.array([covs[p][i] for i in np.arange(Nx)]),axis=0).T
            cov_derivs[p]/=x_grad
#             print(cov_derivs[p].shape,x_grad.shape)
            cov_derivs[p]=cov_derivs[p][:,:,np.int(Nx/2)]
    
    cov_inv=np.linalg.inv(cov)
    cov_p_inv=np.zeros([ndim]*2)
    i1=0
    for p1 in params:
        i2=0
        for p2 in params:
            cov_p_inv[i1,i2]=np.dot(model_derivs[p1],np.dot(cov_inv,model_derivs[p2]))
            
            if do_cov:
#                 print(cov_p_inv[i1,i2],0.5*np.trace(cov_inv@cov_derivs[p1]@cov_inv@cov_derivs[p2]))
                cov_p_inv[i1,i2]+=0.5*np.trace(cov_inv@cov_derivs[p1]@cov_inv@cov_derivs[p2])
            i2+=1
        i1+=1
    out={}
    out['cov_p']=np.linalg.inv(cov_p_inv)
    out['error']=np.sqrt(np.diag(out['cov_p']))
    out['cov_deriv']=cov_derivs
    out['model_deriv']=model_derivs
    out['cov']=cov
    out['cov_inv']=cov_inv
    out['model']=models[p][1]
    out['x_vars']=x_vars
    kappa_class.do_cov=do_cov0
    return out

In [7]:
#setup parameters
lmax_cl=5000
lmin_cl=2
l0=np.arange(lmin_cl,lmax_cl)

lmin_cl_Bins=lmin_cl+10
lmax_cl_Bins=lmax_cl-10
Nl_bins=40
l_bins=np.int64(np.logspace(np.log10(lmin_cl_Bins),np.log10(lmax_cl_Bins),Nl_bins))
lb=np.sqrt(l_bins[1:]*l_bins[:-1])

l=np.unique(np.int64(np.logspace(np.log10(lmin_cl),np.log10(lmax_cl),Nl_bins*20))) #if we want to use fewer ell

do_cov=True
bin_cl=True

SSV_cov=True
tidal_SSV_cov=False

bin_xi=True
theta_bins=np.logspace(np.log10(1./60),1,20)

In [8]:
zs_bins=lsst_source_tomo_bins()

do_cov=True
bin_cl=True
SSV_cov=True
tidal_SSV=False
do_sample_variance=True
use_window=False

sigma_gamma=0.26
area=14000
f_sky=area*d2r**2/4/np.pi

ns0:  23.376047655582575


/usr/lib/python3.7/site-packages/astropy/units/quantity.py:639: RuntimeWarning: divide by zero encountered in true_divide
  result = super().__array_ufunc__(function, method, *arrays, **kwargs)


In [9]:
f_sky

0.3393695767766752

In [10]:
cov_3X2_kwargs={'l':l0,'do_cov':do_cov,'bin_cl':bin_cl,'l_bins':l_bins,'zg_bins':None,
            'SSV_cov':SSV_cov,'tidal_SSV_cov':tidal_SSV,'do_xi':False,'use_window':use_window,'f_sky':f_sky,'sigma_gamma':sigma_gamma,
            'do_sample_variance':do_sample_variance,'power_spectra_kwargs':{'pk_func':'class_pk'},
            'bin_xi':bin_xi}
zs_bins_kwargs={'zmin':0.3,'zmax':3,'ns0':26,'nbins':3,'z_sigma':0.01}

In [11]:
p=np.atleast_1d(['As','Om','w','wa'])
#p=np.atleast_1d(['s8'])
fishes_zsigma={}
z_sigs=np.logspace(-2,0,10)
for zs in z_sigs:
    zs_bins_kwargs['z_sigma']=zs
    fishes_zsigma[zs]=fisher_calc(params=p,do_cov=False,do_log=False,cov_3X2_kwargs=cov_3X2_kwargs,zs_bins_kwargs=zs_bins_kwargs)
zs_bins_kwargs['z_sigma']=0.05

ns0:  23.376047655582575
[('shear', 'shear')]


/usr/lib/python3.7/site-packages/astropy/units/quantity.py:639: RuntimeWarning: divide by zero encountered in true_divide
  result = super().__array_ufunc__(function, method, *arrays, **kwargs)


6 39 1
[('shear', 'shear')]
6 39 1
[('shear', 'shear')]
6 39 1
[('shear', 'shear')]
6 39 1
[('shear', 'shear')]
6 39 1
[('shear', 'shear')]
6 39 1
[('shear', 'shear')]
6 39 1
[('shear', 'shear')]
6 39 1
[('shear', 'shear')]
6 39 1
[('shear', 'shear')]
6 39 1
[('shear', 'shear')]
6 39 1
[('shear', 'shear')]
6 39 1
[('shear', 'shear')]
6 39 1
ns0:  23.376047655582575
[('shear', 'shear')]


CosmoComputationError: 

Error in Class: spectra_init(L:1297) :error in spectra_pk(pba,ppt,ppm,pnl,psp);
=>spectra_pk(L:2785) :error in spectra_sigma(pba,ppm,psp,8./pba->h,0.,&(psp->sigma8));
=>spectra_sigma(L:2876) :error in spectra_pk_at_k_and_z(pba,ppm,psp,k,z,&pk,pk_ic);
=>spectra_pk_at_k_and_z(L:688) :error in spectra_pk_at_z(pba, psp, logarithmic, z, spectrum_at_z, spectrum_at_z_ic);
=>spectra_pk_at_z(L:418) :error in array_interpolate_spline(psp->ln_tau, psp->ln_tau_size, psp->ln_pk, psp->ddln_pk, psp->ln_k_size, ln_tau, &last_index, output_tot, psp->ln_k_size, psp->error_message);
=>array_interpolate_spline(L:1701) : x=9.560144e+00 > x_max=9.560144e+00

In [ ]:
fig, ax = subplots(1,1, figsize=(15,11.25))
for k in list(fishes_zsigma.keys()):
    sn=1./np.sqrt(np.linalg.det(fishes_zsigma[k]['cov_p'][-2:,-2:]))
    print(k,sn)
    ax.plot(k,sn,'bo')
ax.set_xscale('log')
ax.set_xlabel('$\sigma_z$')
ax.set_ylabel('DETF')
fname='sigmaz_detf'
fig.savefig(fig_home+fname+'.'+fig_format,bbox_inches='tight',format=fig_format)
# yscale('log')

In [ ]:
p=np.atleast_1d(['As','Om','w','wa'])
#p=np.atleast_1d(['s8'])
# fishes_nb={}
zs_bins_kwargs['z_sigma']=0.05
# nbs=[2,10,1,5,3]#10
nbs=np.arange(10)+1
for nb in nbs:
    zs_bins_kwargs['nbins']=nb
    fishes_nb[nb]=fisher_calc(params=p,do_cov=False,do_log=False,cov_3X2_kwargs=cov_3X2_kwargs,zs_bins_kwargs=zs_bins_kwargs)
zs_bins_kwargs['nbins']=3

In [ ]:
fig, ax = subplots(1,1, figsize=(15,11.25))
for k in list(fishes_nb.keys()):
    sn=1./np.sqrt(np.linalg.det(fishes_nb[k]['cov_p'][-2:,-2:]))
    print(k,sn)
    ax.plot(k,sn,'bo')
# xscale('log')
ax.set_xlim(1.5,11)
ax.set_ylim(120,145)
ax.set_xlabel('# tomographic bins')
ax.set_ylabel('DETF')
# ax.set_yscale('log')
fname='nbins_detf'
fig.savefig(fig_home+fname+'.'+fig_format,bbox_inches='tight',format=fig_format)

In [ ]:
p=np.atleast_1d(['As','Om','w','wa'])
#p=np.atleast_1d(['s8'])
fishes_zmax={}
z_ms=[1,1.5,2]
for zm in z_ms:
    zs_bins_kwargs['zmax']=zm
    fishes_zmax[zm]=fisher_calc(params=p,do_cov=False,do_log=False,cov_3X2_kwargs=cov_3X2_kwargs,zs_bins_kwargs=zs_bins_kwargs)
zs_bins_kwargs['zmax']=2

In [ ]:
p=np.atleast_1d(['As','Om','w','wa'])
#p=np.atleast_1d(['s8'])
fishes_zmin={}
z_ms=[0.1,0.3,0.5]
for zm in z_ms:
    zs_bins_kwargs['zmin']=zm
    fishes_zmin[zm]=fisher_calc(params=p,do_cov=False,do_log=False,cov_3X2_kwargs=cov_3X2_kwargs,zs_bins_kwargs=zs_bins_kwargs)
zs_bins_kwargs['zmin']=0.3

In [ ]:
fig, ax = subplots(1,1, figsize=(15,11.25))
for k in list(fishes_zmax.keys()):
    sn=1./np.sqrt(np.linalg.det(fishes_zmax[k]['cov_p'][-2:,-2:]))
    print(k,sn)
    ax.plot(k,sn,'bo')
# xscale('log')
# ax.set_xlim(1.5,11)
# ax.set_ylim(120,145)
ax.set_xlabel('$z_{max}$')
ax.set_ylabel('DETF')
# ax.set_yscale('log')
fname='zmax_detf'
fig.savefig(fig_home+fname+'.'+fig_format,bbox_inches='tight',format=fig_format)

In [ ]:
fig, ax = subplots(1,1, figsize=(15,11.25))
for k in list(fishes_zmin.keys()):
    sn=1./np.sqrt(np.linalg.det(fishes_zmin[k]['cov_p'][-2:,-2:]))
    print(k,sn)
    ax.plot(k,sn,'bo')
# xscale('log')
# ax.set_xlim(1.5,11)
# ax.set_ylim(120,145)
ax.set_xlabel('$z_{min}$')
ax.set_ylabel('DETF')
# ax.set_yscale('log')
fname='zmin_detf'
fig.savefig(fig_home+fname+'.'+fig_format,bbox_inches='tight',format=fig_format)

In [40]:
1./np.sqrt(np.linalg.det(fish['cov_p'][-2:,-2:]))

129.55943638832505

In [104]:
fish['error']

array([6.82122628e-11, 4.63920659e-03, 6.13856560e-02, 3.32017991e-01])

In [105]:
fish['error']

array([6.82122628e-11, 4.63920659e-03, 6.13856560e-02, 3.32017991e-01])

In [106]:
m=fish['model']
ci=fish['cov_inv']
np.sqrt(np.dot(m,np.dot(ci,m)))

591.9045764062098

In [107]:
np.linalg.eig(fish['cov_p'])

(array([1.13065188e-01, 8.07959887e-20, 9.47637330e-04, 1.28418598e-05]),
 array([[ 1.13460273e-10, -1.00000000e+00, -1.40431085e-09,
         -1.01453293e-08],
        [ 6.32557140e-03, -1.02154712e-08,  6.66841906e-02,
          9.97754081e-01],
        [-1.58727538e-01,  6.87184098e-10, -9.85057221e-01,
          6.68419074e-02],
        [ 9.87302160e-01,  2.90847035e-10, -1.58793862e-01,
          4.35356764e-03]]))